In [1]:
import os
import cobra
import dnngior
os.environ['GRB_LICENSE_FILE'] = '../../content/licenses/gurobi.lic'

Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
The rest of dnngior features can be used without it.


In [13]:
import sys
import os
from cobra.io import read_sbml_model

sys.path.append(os.path.abspath("../../src"))

model = read_sbml_model("../../models/fixed_models/MAG099_gapfilled_noO2.xml")


print(f"模型中包含 {len(model.reactions)} 个反应，{len(model.metabolites)} 个代谢物，{len(model.genes)} 个基因。")
reaction_ids = [rxn.id for rxn in model.reactions if "00141" in rxn.id]
print(reaction_ids)

模型中包含 999 个反应，983 个代谢物，619 个基因。
['EX_cpd00141_e0', 'TR_cpd00141_e0']


In [15]:
gapfilled_model_complete = dnngior.Gapfill(draftModel = "../../models/fixed_models/MAG099_gapfilled_noO2.xml", 
                                          medium_file = '../../models/complete_media_without_oxygen.tsv', 
                                          objectiveName = 'EX_cpd00141_e0')

Gap-filling database =  ModelSEED


No objective coefficients in model. Unclear what should be optimized


Loading medium from: ../../models/complete_media_without_oxygen.tsv
#reactions not found in NN-keys:  117 / 933
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Flux through biomass reaction is 1.00000000
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  76605 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  38302 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  19151 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  9575 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  4787 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  2393 


Warning for adding constraints: zer

In [20]:
print("New reactions:", len(gapfilled_model_complete.added_reactions))
cobra_model = gapfilled_model_complete.gapfilledModel
print(" Added reactions:")
for reaction_id in gapfilled_model_complete.added_reactions:
    rxn = cobra_model.reactions.get_by_id(reaction_id) 
    print(f"{rxn.id}: {rxn.name} — {rxn.reaction}")


New reactions: 7
 Added reactions:
rxn24523_c0: propionyl-CoA hydrolase — cpd00001_c0 + cpd00086_c0 --> cpd00010_c0 + cpd00067_c0 + cpd00141_c0
rxn03130_c0: UDP-2,3-bis[(3R)-3-hydroxymyristoyl]-alpha-D-glucosamine 2,3-bis[(3R)-3-hydroxymyristoyl]-beta-D-glucosaminyl 1-phosphate phosphohydrolase — cpd00001_c0 + cpd02835_c0 --> 2.0 cpd00067_c0 + cpd00091_c0 + cpd02930_c0
rxn00677_c0: Acetyl-CoA:propanoate CoA-transferase — cpd00022_c0 + cpd00141_c0 <=> cpd00029_c0 + cpd00086_c0
rxn14178_c0: 2-oxobutanoate:ferredoxin 2-oxidoreductase (CoA-propanoylating) — cpd00010_c0 + cpd00094_c0 + 2.0 cpd11621_c0 <=> cpd00011_c0 + cpd00067_c0 + cpd00086_c0 + 2.0 cpd11620_c0
rxn00250_c0: Pyruvate:carbon-dioxide ligase (ADP-forming) — cpd00002_c0 + cpd00020_c0 + cpd00242_c0 --> cpd00008_c0 + cpd00009_c0 + cpd00032_c0 + cpd00067_c0
EX_cpd00141_e0: EX_cpd00141_e0 — cpd00141_e0 --> 
rxn08024_c0: alpha-amylase (periplasm) — cpd11735_c0 <=> cpd01329_c0


In [21]:
new_rxn_ids = gapfilled_model_complete.added_reactions

for rxn_id in new_rxn_ids:
    if rxn_id not in model.reactions:
        new_rxn = cobra_model.reactions.get_by_id(rxn_id).copy()
        model.add_reactions([new_rxn])
        print(f"Added: {new_rxn.id}")


Added: rxn24523_c0
Added: rxn03130_c0
Added: rxn00677_c0
Added: rxn14178_c0
Added: rxn00250_c0
Added: rxn08024_c0


In [22]:
model.objective = 'EX_cpd00141_e0'
solution = model.optimize()
print("flux：", solution.objective_value)

flux： 1000.0


In [23]:
from cobra.io import write_sbml_model


write_sbml_model(model, "../../models/fixed_model_V2/MAG099_gapfilled_noO2.xml")
print("saved")

saved
